
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Fall 2024<br>
<font color=3C99D size=5>
Practical Assignment 1 - Local Search <br>
<font color=696880 size=4>
Davoud Kareshki


____

# Personal Data

In [1]:
# Set your student number
student_number = '401109477'
Name = 'Sadegh'
Last_Name = 'Mohammadian'

# Problem definition
In this homework, you should implement **Hill-Climbing** and **Simulated-Annealing** algorithms.

Suppose we have a graph with 20 nodes and an edge between each pair of nodes. We aim to pick 5 nodes from the graph that induced subgraph of these nodes has a high sum of edges. To do that we will implement 3 algorithms. First for finding the actual answer with  **naive(brute force)** solution. Second, you should implement  **Hill-Climbing**, and then you should implement **Simulated-Annealing**.

# Rules
- Uncompleted functions are to make your coding easier. You are allowed to change the format of codes and functions or add a function by yourself. But you should implement **Hill-Climbing** and  **Simulated-Annealing**.
- Run all cells and leave the outputs
- Half of the point of this problem is for implementing **Hill-Climbing** correctly and the other half is for implementing **Simulated-Annealing** correctly.

In [71]:
import numpy as np
import random
import math

# Generator
Generator, generate a graph with n nodes. edges matrix is an n*n matrix that edges[i][j] is the weight of the edge between nodes i and j.

In [72]:
def Generator(n, seed=42) :
    random.seed(seed)
    edges = np.zeros((n,n))
    for i in range(n) :
        for j in range(n) :
            if i >= j :
                continue
            edges[i][j] = random.randint(0,100)
            edges[j][i] = edges[i][j]
    return edges

In [73]:
n = 20
edges = Generator(n)

In [74]:
print(edges)

[[ 0. 81. 14.  3. 94. 35. 31. 28. 17. 94. 13. 86. 94. 69. 11. 75. 54.  4.
   3. 11.]
 [81.  0. 27. 29. 64. 77.  3. 71. 25. 91. 83. 89. 69. 53. 28. 57. 75. 35.
   0. 97.]
 [14. 27.  0. 20. 89. 54. 43. 35. 19. 27. 97. 43. 13. 11. 48. 12. 45. 44.
  77. 33.]
 [ 3. 29. 20.  0.  5. 93. 58. 68. 15. 48. 10. 70. 37. 80. 79. 46. 73. 24.
  90.  8.]
 [94. 64. 89.  5.  0.  5. 84. 29. 98. 37. 10. 29. 12. 48. 35. 58. 81. 46.
  20. 47.]
 [35. 77. 54. 93.  5.  0. 45. 26. 85. 34. 89. 87. 82.  9. 77. 81. 21. 68.
  93. 31.]
 [31.  3. 43. 58. 84. 45.  0. 20. 59. 48. 34. 81. 88. 71. 28. 87. 41. 98.
  99.  7.]
 [28. 71. 35. 68. 29. 26. 20.  0. 29.  4. 40. 51. 34.  8. 27. 72. 91. 40.
  27. 83.]
 [17. 25. 19. 15. 98. 85. 59. 29.  0. 63. 50. 82. 58. 18. 33. 17. 31. 95.
  71. 68.]
 [94. 91. 27. 48. 37. 34. 48.  4. 63.  0. 33. 95. 74. 54. 74. 51. 46. 28.
  17. 65.]
 [13. 83. 97. 10. 10. 89. 34. 40. 50. 33.  0. 63. 11. 96.  6. 14. 19. 80.
  20. 87.]
 [86. 89. 43. 70. 29. 87. 81. 51. 82. 95. 63.  0. 54. 76.  8. 49.

# Brute force

In [75]:
def brute_force(n, k, edges, picked = set()) :
    '''
        input
            n = number of nodes
            k = number of nodes to pick
            picked = already picked nodes
        output
            ans = best answer that contains the nodes of 'picked'
    '''
    n = edges.shape[0]
    if len(picked) == k :
        ans = 0
        for u in picked :
            for v in picked :
                if v > u :
                    ans += edges[u][v]
        return ans

    ans = 0
    for v in range(n) :
        if v not in picked :
             picked.add(v)
             ans = max(ans, brute_force(n, k, edges, picked))
             picked.remove(v)
    return ans

In [76]:
k = 5
ans = brute_force(n, k, edges)
print(ans)

879.0


# Hill climbing

In [77]:
def random_choice(n, k):
    '''
        choose k random unique number between 1 to n
    '''
    output = []
    output = random.sample(range(0,n), k)
    return output

In [78]:
def get_value(state, edges):
    '''
        state is a list that contains some nodes
        return sum of edges of state nodes
    '''
    ans = 0
    for i in range(len(state)):
      for j in range(i+1,len(state)):
        ans += edges[state[i]][state[j]]
    return ans

We consider two states like X and Y as neighbor states, if there is exactly one node in X that isn't in Y and there is exactly one node in Y that isn't in X.

For instance [2,3,4,5,6] and [2,3,4,6,7] are neighbors but [2,3,4,5,6] and [1,2,3,7,8] aren't neighbors.

In [79]:
def get_neighbors(n, state):
    '''
        return neighbors of state
    '''
    neighbors = []

    state.sort()
    copy_state = state.copy()
    for i in range(len(state)):
      for j in range(n):
        if j not in copy_state:
          copy_state[i]=j
          neighbors.append(copy_state)
          copy_state = state.copy()

    return neighbors


In [80]:
def hill_climbing(n, k, edges, num_iters = 10000):
    '''
        input
            n = number of graph nodes
            k = number of nodes to pick
            edges = graph edges weights
            num_iters = maximum number of iterations
        output
            best_value = best state value
    '''

    current_state = random_choice(n,k)
    current_value = get_value(current_state, edges)

    while num_iters > 0 :
        num_iters -= 1

        neighbors = get_neighbors(n,current_state)

        value = []
        for i in range(len(neighbors)):
          value.append(get_value(neighbors[i],edges))
        max_value = max(value)
        max_index = value.index(max_value)

        if(max_value >= current_value):
          current_state = neighbors[max_index]
          current_value = max_value

    best_value = current_value

    return best_value

In [82]:
ans = hill_climbing(n, k, edges)
print(ans)

879.0


# Simulated annealing

In [84]:
def simulated_annealing(n, k, edges, alpha = 0.9, initial_temp = 100000, max_iteration = 100000, min_temperature = 0.0001):
    '''
    input
        alpha is the temperature decay rate
        max_iteration is the maximum number of iteration (termination condition)
        min_temperature is the minimum temperature (termination condition)
    output
        best state value
    '''
    temperature = initial_temp
    best_value = float('-INF')


    current_state = random_choice(n, k)
    while(temperature >= min_temperature and max_iteration > 0):
        max_iteration -= 1
        nextState = random.choice(get_neighbors(n, current_state))
        dE = get_value(nextState, edges) - get_value(current_state, edges)
        if(dE > 0):
          current_state = nextState
        else:
          probability = math.exp(dE / temperature)
          random_element = random.choices([0, 1] , [1-probability , probability])[0]
          if(random_element == 1):
            current_state = nextState

        temperature = alpha * temperature
    best_value = get_value(current_state, edges)
    return best_value


In [100]:
ans = simulated_annealing(n, k, edges)
print(ans)

879.0


# How much Hill climbing and Simulated annealing answers are close to actual answers?

Run 3 algorithms 10 times and watch the result. Hill climbing and simulated annealing results should be close to actual answers(brute force results).

Running the following cell may take around 2 minutes.

In [101]:
k = 5
seeds = [10, 20, 30, 40, 50, 60, 70, 80, 142, 2024]
brute_force_result = []
hill_climbing_result = []
simulated_annealing_result = []
for seed in seeds:
    edges = Generator(n, seed)
    brute_force_result.append(brute_force(n, k, edges))
    hill_climbing_result.append(hill_climbing(n, k, edges))
    simulated_annealing_result.append(simulated_annealing(n, k, edges))


In [102]:
print('brute_force_result         :', brute_force_result)
print('hill_climbing_result       :', hill_climbing_result)
print('simulated_annealing_result :', simulated_annealing_result)

brute_force_result         : [785.0, 810.0, 819.0, 783.0, 832.0, 789.0, 813.0, 763.0, 818.0, 860.0]
hill_climbing_result       : [724.0, 702.0, 819.0, 742.0, 832.0, 732.0, 798.0, 761.0, 781.0, 860.0]
simulated_annealing_result : [785.0, 781.0, 812.0, 724.0, 689.0, 711.0, 813.0, 735.0, 818.0, 838.0]
